### steps
1. Create an sample SQL database
2. create config.ini for Mysql database connection
3. create table
4. performing SQL on database to solve problem

In [2]:
from mysql.connector import MySQLConnection, Error
import mysql
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [3]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


In [4]:
def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(query_insert, table_values):
    """execute query to insert values to table
    :query_table:: to create scheme table
    :table_values:: to be inserted table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('creating MYSQL table...')
        cursor.executemany(query_insert, table_values)

        conn.commit()
        print('MYSQL table created')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

In [5]:
def main():
    query_create = "CREATE TABLE Logs (Id INTEGER PRIMARY KEY, Num VARCHAR(30));"
    query_insert = "INSERT INTO Logs (Id,Num) VALUES(%s,%s)"
    #change Id integer and Num values for every run, to get distinct Id
    logs = [(1, '1'),
            (2, '1'),
            (3, '1'),
            (4, '2'), 
            (5, '1'),
            (6, '2'),
            (7, '2'),
            (8, '2'),
            (9, '3'),
            (10, '3')]
    create_msql_table(query_create)
    insert_msql_table(query_insert, logs)

if __name__ == '__main__':
    main()

Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'logs' already exists
Connecting to MYSQL database...
Connected to MYSQL database
creating MYSQL table...
Error:  1062 (23000): Duplicate entry '1' for key 'logs.PRIMARY'


In [7]:
query_fetching_consecutive_number = "select distinct Num as ConsecutiveNums \
                                    from ( \
                                        select Id, Num, \
                                            lag(Num) over (order by Id) as prev, \
                                            lead(Num) over (order by Id) as next \
                                        from Logs ) temp \
                                    where Num = prev and prev = next"

try:
    dbconfig = read_db_config()
    conn = MySQLConnection(**dbconfig)
    cursor = conn.cursor()
    cursor.execute(query_fetching_consecutive_number)

    row = cursor.fetchone()

    while row is not None:
        print(row)
        row = cursor.fetchone()

except Error as e:
    print(e)

finally:
    cursor.close()
    conn.close()

('1',)
('2',)


In [6]:
!mysql -u root --password=root python_mysql -e "{query_fetching_consecutive_number}"

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
| 2               |
+-----------------+


In [7]:
!mysql -u root --password=root python_mysql -e "select * from Logs"

mysql: [Warning] Using a password on the command line interface can be insecure.
+----+------+
| Id | Num  |
+----+------+
|  1 | 1    |
|  2 | 1    |
|  3 | 1    |
|  4 | 2    |
|  5 | 1    |
|  6 | 2    |
|  7 | 2    |
|  8 | 2    |
|  9 | 3    |
| 10 | 3    |
+----+------+
